In [ ]:
import sys
sys.stdout.flush()

# Or force flush after prints
print("Your message", flush=True)

import os
import sys
import numpy as np
import pandas as pd
# At the top of your notebook
%matplotlib inline
import matplotlib.pyplot as plt

import cv2
from pathlib import Path
from datetime import datetime
import math

import seaborn as sns
import json

import socket

import tensorflow as tf
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import (
    Flatten, Dropout, Dense, Input, GlobalAveragePooling2D, Conv2D,
    BatchNormalization, Activation, MaxPooling2D, Lambda
)
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import (
    ModelCheckpoint, EarlyStopping, ReduceLROnPlateau, LearningRateScheduler
)
from tensorflow.keras.applications import MobileNetV2, EfficientNetB0

from sklearn.metrics import confusion_matrix, classification_report, precision_recall_fscore_support
from sklearn.utils.class_weight import compute_class_weight

# Enable mixed precision
from tensorflow.keras import mixed_precision

print("✅ All libraries imported successfully")


In [ ]:
def get_hostname():
    
    print("Available devices:")
    print(tf.config.list_physical_devices())
    hostname = socket.gethostname()
    
    return hostname

In [ ]:
# Check all available devices

# Check GPU specifically
gpus = tf.config.list_physical_devices('GPU')
print(f"GPUs found: {len(gpus)}")
for gpu in gpus:
    print(f"  {gpu}")

if get_hostname() == "xscape7x":
    print("🔄 Snapdragon X Elite detected - optimizing for CPU")
    
    # Use environment variables instead (set before TF init)
    os.environ['TF_ENABLE_ONEDNN_OPTS'] = '1'
    os.environ['OMP_NUM_THREADS'] = '8'
    os.environ['TF_NUM_INTEROP_THREADS'] = '8'
    os.environ['TF_NUM_INTRAOP_THREADS'] = '8'
    
    # Restart Python kernel after setting these
    print("✅ Environment configured - restart kernel for full effect")
else:
    # Standard GPU check
    if tf.config.list_physical_devices('GPU'):
        mixed_precision.set_global_policy('mixed_float16')
    else:
        mixed_precision.set_global_policy('float32')

# === NOW SAFE TO USE TENSORFLOW ===
print("Available devices:", tf.config.list_physical_devices())
print("GPUs found:", len(tf.config.list_physical_devices('GPU')))
print("Final policy:", mixed_precision.global_policy())

In [ ]:


# =========================================================================
# Clean Configuration System
# =========================================================================

class ModelConfig:
    """Clean configuration class with backbone-specific presets"""
    
    BACKBONE_PRESETS = {
        'custom_cnn': {
            'picture_size': 48,
            'color_mode': 'grayscale',
            'batch_size': 64,
            'epochs': 50,
            'learning_rate': 0.001,
            'dropout_rate': 0.25,
            'dense_units': [512],
            'aug_level': 'light',
            'use_class_weights': True,
            'use_lr_schedule': True,
            'weight_decay':1e-4
        },
        'mobilenet': {
            'picture_size': 96,
            'color_mode': 'rgb',
            'batch_size': 32,
            'epochs': 30,
            'learning_rate': 0.0001,
            'dropout_rate': 0.3,
            'dense_units': [512, 256],
            'aug_level': 'strong',
            'use_class_weights': True,
            'use_lr_schedule': True,
            'fine_tune': True,
            'fine_tune_epochs': 20,
            'fine_tune_unfreeze_layers': 30,
            'weight_decay':1e-4
        },
        'efficientnet': {
            'picture_size': 128,
            'color_mode': 'rgb',
            'batch_size': 16,
            'epochs': 40,
            'learning_rate': 0.0001,
            'dropout_rate': 0.4,
            'dense_units': [1024, 512],
            'aug_level': 'strong',
            'use_class_weights': True,
            'use_lr_schedule': True,
            'fine_tune': True,
            'fine_tune_epochs': 25,
            'fine_tune_unfreeze_layers': 50,
            'weight_decay':1e-4
        }
    }
    
    AUGMENTATION_LEVELS = {
        'none': dict(rescale=1./255),
        'light': dict(
            rescale=1./255,
            width_shift_range=0.1,
            height_shift_range=0.1,
            zoom_range=0.2,
            horizontal_flip=True
        ),
        'strong': dict(
            rescale=1./255,
            rotation_range=20,
            width_shift_range=0.2,
            height_shift_range=0.2,
            shear_range=0.15,
            zoom_range=0.2,
            brightness_range=[0.7, 1.3],
            horizontal_flip=True,
            fill_mode='nearest'
        ),
    }
    
    def __init__(self, backbone='mobilenet'):
        """Initialize with backbone preset"""
        if backbone not in self.BACKBONE_PRESETS:
            raise ValueError(f"Backbone must be one of {list(self.BACKBONE_PRESETS.keys())}")
        
        self.backbone = backbone
        self.config = self.BACKBONE_PRESETS[backbone].copy()
        self.config['backbone'] = backbone
        self.config['no_of_classes'] = 7
        
    def get(self, key, default=None):
        """Get configuration value"""
        return self.config.get(key, default)
    
    def set(self, key, value):
        """Set configuration value"""
        self.config[key] = value
    
    def to_dict(self):
        """Return full configuration"""
        return self.config


In [ ]:

# =========================================================================
# Data Loading Utilities
# =========================================================================
def is_on_kaggle():
    """Detect if running on Kaggle."""
    return os.path.exists('/kaggle/input')

def get_data_path():
    """Detect environment and return appropriate data path."""
    if is_on_kaggle():
        print("🌐 Running on Kaggle")
        import kagglehub
        image_path = kagglehub.dataset_download("jonathanoheix/face-expression-recognition-dataset")
        folder_path = os.path.join(image_path, "images")
    else:
        print("💻 Running on local machine")
        folder_path = "data/images/"
    
    return folder_path


In [ ]:

def create_data_generators(config, data_path):
    """Create clean data generators"""
    
    # Get augmentation parameters
    aug_params = config.AUGMENTATION_LEVELS[config.get('aug_level', 'light')]
    
    # Handle preprocessing based on backbone
    if config.backbone in ['mobilenet', 'efficientnet']:
        from tensorflow.keras.applications.mobilenet_v2 import preprocess_input
        aug_params = aug_params.copy()
        aug_params['preprocessing_function'] = preprocess_input
    
    # Create generators
    train_gen = ImageDataGenerator(**aug_params)
    val_gen = ImageDataGenerator(**aug_params)
    
    # Determine color mode
    color_mode = config.get('color_mode', 'grayscale')
    
    train_set = train_gen.flow_from_directory(
        os.path.join(data_path, "train"),
        target_size=(config.get('picture_size'), config.get('picture_size')),
        color_mode=color_mode,
        batch_size=config.get('batch_size'),
        class_mode='categorical',
        shuffle=True,
    )
    
    validation_set = val_gen.flow_from_directory(
        os.path.join(data_path, "validation"),
        target_size=(config.get('picture_size'), config.get('picture_size')),
        color_mode=color_mode,
        batch_size=config.get('batch_size'),
        class_mode='categorical',
        shuffle=False,
    )
    
    print("Train image shape:", train_set.image_shape)
    print("Val image shape:  ", validation_set.image_shape)

    print(f"\n✅ Data loaded successfully")
    print (f"Colour Mode: {color_mode}")
    print(f"Training samples: {train_set.n}")
    print(f"Validation samples: {validation_set.n}")
    print(f"Class indices: {train_set.class_indices}")
    
    return train_set, validation_set



In [ ]:

# =========================================================================
# Model Building
# =========================================================================

def build_custom_cnn(config):
    """Build custom CNN model"""
    picture_size = config.get('picture_size')
    color_mode = config.get('color_mode')
    channels = 1 if color_mode == 'grayscale' else 3
    input_shape=(picture_size,picture_size,channels)
    dropout_rate = config.get('dropout_rate')
    
    model = Sequential()
   # Block 1 - Double Conv
    model.add(Conv2D(filters=32, 
                        kernel_size=(3, 3), 
                        padding='same', 
                        input_shape=input_shape,
                        name='conv2d_1'))

    # Block 2 - Double Conv
    model.add(Conv2D(filters=128, 
                      kernel_size=(3, 3), 
                      padding='same',
                      name='conv2d_2'))
    model.add(BatchNormalization(name='bn_2'))
    model.add(Activation('relu'))
    model.add(MaxPooling2D(pool_size=(2, 2),name='maxpool2d_2'))
    model.add(Dropout(dropout_rate,name='dropout_2'))
    
    # Block 3 - Double Conv
    model.add(Conv2D(filters=512, 
                      kernel_size=(5, 5), 
                      padding='same',
                      name='conv2d_3'))
    model.add(BatchNormalization(name='bn_3'))
    model.add(Activation('relu'))
    model.add(MaxPooling2D(pool_size=(2, 2),name='maxpool2d_3'))
    model.add(Dropout(dropout_rate,name='dropout_3'))
    
    # Block 4 - Double Conv
    model.add(Conv2D(filters=512, 
                      kernel_size=(3, 3), 
                      padding='same',
                      name='conv2d_4'))
    model.add(BatchNormalization(name='bn_4'))
    model.add(Activation('relu'))
    model.add(MaxPooling2D(pool_size=(2, 2),name='maxpool2d_4'))
    model.add(Dropout(dropout_rate,name='dropout_4'))

    
    # Block 5 - Double Conv
    model.add(Conv2D(filters=512, 
                      kernel_size=(3, 3), 
                      padding='same',
                      name='conv2d_5'))
    model.add(BatchNormalization(name='bn_5'))
    model.add(Activation('elu'))
    model.add(MaxPooling2D(pool_size=(2, 2),name='maxpool2d_5'))
    model.add(Dropout(dropout_rate,name='dropout_5'))
 
        
    model.add(Flatten())
    
    # Add dense layers
    for units in config.get('dense_units', [512]):
        model.add(Dense(units, activation='relu'))
        model.add(BatchNormalization())
        model.add(Dropout(config.get('dropout_rate')))
    
    model.add(Dense(config.get('no_of_classes'), activation='softmax', dtype='float32'))
    
    return model, None


In [ ]:

def build_transfer_model(config, backbone_class):
    """Build transfer learning model"""
    picture_size = config.get('picture_size')
    color_mode = config.get('color_mode')
    channels = 1 if color_mode == 'grayscale' else 3
    
    inputs = Input(shape=(picture_size, picture_size, channels))
    
    # Convert grayscale to RGB if needed
    if channels == 1:
        x = Lambda(lambda z: tf.image.grayscale_to_rgb(z))(inputs)
    else:
        x = inputs
    
    # Load pre-trained backbone
    base_model = backbone_class(
        weights='imagenet',
        include_top=False,
        input_shape=(picture_size, picture_size, 3)
    )
    
    base_model.trainable = False
    x = base_model(x, training=False)
    x = GlobalAveragePooling2D()(x)
    x = Dropout(config.get('dropout_rate'))(x)
    
    # Add dense layers
    for units in config.get('dense_units', [512]):
        x = Dense(units, activation='relu')(x)
        x = Dropout(config.get('dropout_rate'))(x)
    
    outputs = Dense(config.get('no_of_classes'), activation='softmax', dtype='float32')(x)
    
    model = Model(inputs, outputs)
    return model, base_model


In [ ]:

def build_model(config):
    """Build model based on configuration"""
    backbone = config.backbone
    
    if backbone == 'custom_cnn':
        return build_custom_cnn(config)
    elif backbone == 'mobilenet':
        return build_transfer_model(config, MobileNetV2)
    elif backbone == 'efficientnet':
        return build_transfer_model(config, EfficientNetB0)
    else:
        raise ValueError(f"Unknown backbone: {backbone}")


In [ ]:

# =========================================================================
# Training Utilities
# =========================================================================

def create_callbacks(config):
    """Create training callbacks"""
    callbacks = [
        ModelCheckpoint('best_model.keras', monitor='val_accuracy', save_best_only=True),
        EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True),
        ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=2, min_delta=0.0001)
    ]
    
    if config.get('use_lr_schedule'):
        def cosine_annealing(epoch, lr):
            initial_lr = float(config.get('learning_rate'))
            min_lr = 1e-7
            warmup = 5
            total_epochs = config.get('epochs')
            
            if epoch < warmup:
                return initial_lr * (epoch + 1) / warmup
            
            progress = (epoch - warmup) / (total_epochs - warmup)
            lr_out = min_lr + (initial_lr - min_lr) * 0.5 * (1 + math.cos(math.pi * progress))
            return max(lr_out, min_lr)
        
        callbacks.append(LearningRateScheduler(cosine_annealing))
    
    return callbacks

def compute_class_weights(train_set):
    """Compute class weights for imbalanced data"""
    class_weights = compute_class_weight(
        'balanced',
        classes=np.unique(train_set.classes),
        y=train_set.classes
    )
    return dict(enumerate(class_weights))


<h1> Run Training </h1>

In [ ]:


def set_host_config_overides(config, dev=True):
    
    hostname = get_hostname() 

    if  hostname == "xscape7x": # local machine
        print("Hostname:", hostname)
        print("Overiding settings for xscape7x...")
        # Override any settings
        if dev==True: 
            config.set("epochs", 1)
            config.set("batch_size", 32)
            config.set("fine_tune_epochs", 1)
            config.set("fine_tune_unfreeze_layers", 1)
        
    else:                                           # run on remote server
        pass  # No overrides

    print("=" * 50)
    print(f"Configuration: {config.backbone.upper()}")
    print("=" * 50)
    for key, value in config.config.items():
        print(f"{key:20}: {value}")
    print("=" * 50)

    return 


In [ ]:
def create_output_dir():
    # - create output folder and provide output path
    timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")

    if os.path.exists('/kaggle/input'):
        output_dir = "/kaggle/working/" + timestamp + "/"  
    else:
        output_dir = "outputs/" + timestamp + "/"

    os.makedirs(output_dir, exist_ok=True)

    print (f"Output directory: {output_dir}")
    return output_dir

In [ ]:
def save_model(model, model_name_prefix):
    model_name = f"{model_name_prefix}_{timestamp}.keras"
    out_path = Path(output_dir) / model_name
    try:
        model.save(str(out_path))
        print(f"\n💾 Model saved: {out_path}")
    except Exception as e:
        print(f"Error saving model: {e}")
    
    return out_path

In [ ]:
# =========================================================================
# Train the Model Stage 1
# =========================================================================
# Class weights
def train_stage1(config, model,train_set, validation_set, class_weights, callbacks):

    # Training
    print(f"Training {config.backbone} model...")
    print("\n" + "="*70)
    print("🚀 STARTING TRAINING")
    print("="*70)
    print(f"Target: {config.get('epochs')} epochs with early stopping")
    print(f"Batch size: {config.get('batch_size')}")
    print(f"Learning rate: {config.get('learning_rate')}")
    print(f"Augmentation: {config.get('aug_level')}")
    print(f"Backbone: {config.get('backbone')}")
    print(f"Class weights: {'Enabled' if class_weights else 'Disabled'}")
    print("="*70)
    
    
    # Stage 1: Train head
    history_head = model.fit(
        train_set,
        epochs=config.get("epochs"),
        validation_data=validation_set,
        callbacks=callbacks,
        class_weight=class_weights,
        verbose=1,
    )
    return history_head

In [ ]:
# =========================================================================
# Train the Model Stage 2 - Fine Tuning
# =========================================================================

def train_stage2(config, base_model, model, train_set, validation_set, class_weights, callbacks):
    # Stage 2: Fine-tune (if transfer learning)
    if base_model and config.get("fine_tune"):
        print("Fine-tuning...")

        # Unfreeze layers
        unfreeze_layers = config.get("fine_tune_unfreeze_layers", 30)
        for layer in base_model.layers[unfreeze_layers:]:
            layer.trainable = True

        # Recompile with lower LR
        model.compile(
            optimizer=Adam(learning_rate=config.get("learning_rate") * 0.1),
            loss="categorical_crossentropy",
            metrics=["accuracy"],
        )

        # Continue training
        history_ft = model.fit(
            train_set,
            epochs=config.get("epochs") + config.get("fine_tune_epochs", 0),
            initial_epoch=config.get("epochs"),
            validation_data=validation_set,
            callbacks=callbacks,
            class_weight=class_weights,
            verbose=1,
        )
    return history_ft

<h1>Load Model and Generate Reports</h1>

In [ ]:
def load_model_path():
    import os,re, glob
    from pathlib import Path
    from datetime import datetime
    
     # ensure these are always defined
    latest_ft = None
    latest_hd = None
    latest_dir = None

    # Find all valid .keras files
    valid_files = [
        f for f in glob.glob("outputs/*/*.keras")
        if re.match(r'^\d{8}_\d{6}$', os.path.basename(os.path.dirname(f)))
    ]

    if not valid_files:
        print("❌ No valid .keras files found")
        
    else:
        # Find latest timestamp
        latest = max(valid_files, key=lambda x: datetime.strptime(
            os.path.basename(os.path.dirname(x)), "%Y%m%d_%H%M%S"
        ))
        latest_dir = os.path.dirname(latest)

        # Look for hd and ft variants in the same latest directory
        hd_pattern = os.path.join(latest_dir, "*_hd_*.keras")
        ft_pattern = os.path.join(latest_dir, "*_ft_*.keras")

        hd_files = glob.glob(hd_pattern)
        ft_files = glob.glob(ft_pattern)

        latest_hd = hd_files[0] if hd_files else None
        latest_ft = ft_files[0] if ft_files else None

        print(f"Latest HD: {latest_hd}")
        print(f"Latest FT: {latest_ft}")
    
    return latest_ft, latest_hd, latest_dir

In [ ]:
def load_models_from_file(path_hd, path_ft):
    from tensorflow.keras.models import load_model
    model_hd=load_model(path_hd)
    model_ft=load_model(path_ft)

    return model_hd, model_ft


In [ ]:

# After training
# history_head, history_ft = train_two_stage(...)
def plot_training_curve_head(train_set, val_acc, train_loss, val_loss):
    print ("plot both stages combined (recommended so plots show full training curve):")
    # 1) Get scalar evaluation results from the model (safe)


    # Plot
    fig, axes = plt.subplots(1, 2, figsize=(15, 5))
    axes[0].plot(train_set, label='Train Accuracy', linewidth=2)
    axes[0].plot(val_acc, label='Val Accuracy', linewidth=2)
    axes[0].set_title('Model Accuracy'); axes[0].legend(); axes[0].grid(True)

    axes[1].plot(train_loss, label='Train Loss', linewidth=2)
    axes[1].plot(val_loss, label='Val Loss', linewidth=2)
    axes[1].set_title('Model Loss'); axes[1].legend(); axes[1].grid(True)

    plt.tight_layout()
   
    #print(f"📊 Final Validation Accuracy: {val_acc:.4f} ({val_acc*100:.2f}%)")
    return plt



In [ ]:
# =======================================================================
# Training History Visualization
# =======================================================================
def plot_training_history(history_hd, history_ft):
    # 1) Get scalar evaluation results from the model (safe)
    # train_loss, train_acc = model.evaluate(train_set, verbose=0)
    # val_loss, val_acc = model.evaluate(validation_set, verbose=0)

    # 2) Consolidate histories (head + fine-tune) for plotting if present
    def _get_hist_list(h):
        return h.history if h is not None else {}

    head_hist = history_hd if 'history_head' in globals() else None
    ft_hist = history_ft if 'history_ft' in globals() else None

    # Combine metrics safely
    def _concat_metric(metric):
        vals = []
        if head_hist is not None:
            vals += head_hist.history.get(metric, [])
        if ft_hist is not None:
            vals += ft_hist.history.get(metric, [])
        return vals

    train_acc_list = _concat_metric('accuracy')
    val_acc_list = _concat_metric('val_accuracy')
    train_loss_list = _concat_metric('loss')
    val_loss_list = _concat_metric('val_loss')

    def plot_training_history(history_data):
        """Plot training history with better formatting."""
        fig, axes = plt.subplots(1, 2, figsize=(15, 5))
        
        # Accuracy plot
        axes[0].plot(history_data['accuracy'], label='Train Accuracy', linewidth=2)
        axes[0].plot(history_data['val_accuracy'], label='Val Accuracy', linewidth=2)
        axes[0].set_title('Model Accuracy')
        axes[0].set_xlabel('Epoch')
        axes[0].set_ylabel('Accuracy')
        axes[0].legend()
        axes[0].grid(True, alpha=0.3)
        
        # Loss plot
        axes[1].plot(history_data['loss'], label='Train Loss', linewidth=2)
        axes[1].plot(history_data['val_loss'], label='Val Loss', linewidth=2)
        axes[1].set_title('Model Loss')
        axes[1].set_xlabel('Epoch')
        axes[1].set_ylabel('Loss')
        axes[1].legend()
        axes[1].grid(True, alpha=0.3)
        
        plt.tight_layout()



    # Prepare history data dictionary for plotting
    history_data = {
        'accuracy': train_acc_list,
        'val_accuracy': val_acc_list,
        'loss': train_loss_list,
        'val_loss': val_loss_list
    }

    # Plot the training history
    plot_training_history(history_data)
    
    return plt

In [ ]:
def publish_final_evaluation(train_acc, val_acc, train_loss, val_loss):
    # Final evaluation
    print("\n" + "="*70)
    print("📊 FINAL EVALUATION")
    print("="*70)
    print(f"Training Accuracy:   {train_acc:.4f} ({train_acc*100:.2f}%)")
    print(f"Validation Accuracy: {val_acc:.4f} ({val_acc*100:.2f}%)")
    print(f"Training Loss:       {train_loss:.4f}")
    print(f"Validation Loss:     {val_loss:.4f}")
    print("="*70)
    
    return


In [ ]:
# =======================================================================
# Enhanced Performance Visualizations - confusion matrix
# =======================================================================
# Add this before the visualization section
# before visualization

def generate_confusion_matrix_reports(validation_set, class_labels, out_path):
    #validation_set.reset()
   
    predicted_classes = np.argmax(preds, axis=1)
    true_classes = validation_set.classes
    cm = confusion_matrix(true_classes, predicted_classes)

    # 1. Confusion Matrix
    plt.figure(figsize=(6, 5))
    sns.heatmap(cm, annot=True, fmt='d', cmap='YlOrRd',
                xticklabels=class_labels,
                yticklabels=class_labels,
                cbar_kws={'label': 'Count'},
                linewidths=0.5,
                linecolor='gray')
    plt.title('Confusion Matrix - Emotion Recognition', fontsize=16, fontweight='bold', pad=20)
    plt.xlabel('Predicted Emotion', fontsize=13, fontweight='bold')
    plt.ylabel('True Emotion', fontsize=13, fontweight='bold')
    plt.xticks(rotation=45, ha='right')
    plt.yticks(rotation=0)
    plt.tight_layout()
    plt.savefig(os.path.join(out_path, 'confusion_matrix.png'), dpi=300, bbox_inches='tight')
    plt.show()

    # 2. Normalized Confusion Matrix
    cm_normalized = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]

    plt_norm.figure(figsize=(6, 5))
    sns.heatmap(cm_normalized, annot=True, fmt='.2%', cmap='Blues',
                xticklabels=class_labels,
                yticklabels=class_labels,
                cbar_kws={'label': 'Percentage'})
    plt_norm.title('Normalized Confusion Matrix (%)', fontsize=16, fontweight='bold', pad=20)
    plt_norm.xlabel('Predicted Emotion', fontsize=13, fontweight='bold')
    plt_norm.ylabel('True Emotion', fontsize=13, fontweight='bold')
    plt_norm.xticks(rotation=45, ha='right')
    plt_norm.yticks(rotation=0)
    plt_norm.tight_layout()
    plt_norm.savefig(os.path.join(out_path, 'confusion_matrix_normalized.png'), dpi=300, bbox_inches='tight')
    plt_norm.show()

    # 3. Classification Report
    print("\n" + "="*70)
    print("📈 DETAILED CLASSIFICATION METRICS")
    print("="*70)
    print(classification_report(true_classes, predicted_classes, 
                            target_names=class_labels,
                            digits=4))
    return plt, plt_norm, cm, cm_normalized


In [ ]:



def generate_per_class_report(model, validation_set, class_labels, output_dir):
    # 4. Per-Class Metrics Bar Chart
    
    #validation_set.reset()
    preds = model.predict(validation_set, verbose=0)
    predicted_classes = np.argmax(preds, axis=1)
    true_classes = validation_set.classes
    precision, recall, f1, support = precision_recall_fscore_support(
        true_classes, predicted_classes, labels=range(7)
    )

    fig, ax = plt.subplots(figsize=(7, 5))
    x = np.arange(len(class_labels))
    width = 0.25

    bars1 = ax.bar(x - width, precision, width, label='Precision', color='#3498db')
    bars2 = ax.bar(x, recall, width, label='Recall', color='#2ecc71')
    bars3 = ax.bar(x + width, f1, width, label='F1-Score', color='#e74c3c')

    ax.set_xlabel('Emotion', fontsize=12, fontweight='bold')
    ax.set_ylabel('Score', fontsize=12, fontweight='bold')
    ax.set_title('Per-Emotion Performance Metrics', fontsize=14, fontweight='bold')
    ax.set_xticks(x)
    ax.set_xticklabels(class_labels, rotation=45, ha='right')
    ax.legend()
    ax.grid(axis='y', alpha=0.3)
    ax.set_ylim([0, 1.1])

    # Add value labels on bars
    for bars in [bars1, bars2, bars3]:
        for bar in bars:
            height = bar.get_height()
            ax.text(bar.get_x() + bar.get_width()/2., height,
                    f'{height:.2f}',
                    ha='center', va='bottom', fontsize=8)

    plt.tight_layout()
    return plt
    
def performace_summary(class_labels, precision, recall, f1, support, cm):
    # 5. Summary Table
    summary_df = pd.DataFrame({
        'Emotion': class_labels,
        'Precision': [f'{p:.2%}' for p in precision],
        'Recall': [f'{r:.2%}' for r in recall],
        'F1-Score': [f'{f:.2%}' for f in f1],
        'Support': support.astype(int)
    })

    print("\n" + "="*70)
    print("📊 PERFORMANCE SUMMARY TABLE")
    print("="*70)
    print(summary_df.to_string(index=False))
    print("="*70)

    # 6. Misclassification Analysis
    misclassified = cm.copy()
    np.fill_diagonal(misclassified, 0)

    top_confusions = []
    for i in range(7):
        for j in range(7):
            if i != j:
                top_confusions.append((class_labels[i], class_labels[j], misclassified[i, j]))

    top_confusions.sort(key=lambda x: x[2], reverse=True)

    print("\n" + "="*70)
    print("⚠️  TOP 5 MISCLASSIFICATION PAIRS")
    print("="*70)
    for true_label, pred_label, count in top_confusions[:5]:
        print(f"{true_label:>10} → {pred_label:<10} : {count:>4} times")
    print("="*70)
    
    return
    

In [ ]:

# =======================================================================
# Sample Predictions Visualization
# =======================================================================

def test_random_predictions(model, validation_set, class_labels):

    import random

    # Get a random batch from validation set
    random_batch_idx = random.randint(0, len(validation_set) - 1)
    sample_images, sample_labels = validation_set[random_batch_idx]

    # Predict
    sample_preds = model.predict(sample_images, verbose=0)
    sample_pred_classes = np.argmax(sample_preds, axis=1)
    sample_true_classes = np.argmax(sample_labels, axis=1)

    # Randomly select 16 indices from the batch
    num_samples = min(16, len(sample_images))
    random_indices = random.sample(range(len(sample_images)), num_samples)

    # Visualize predictions
    fig, axes = plt.subplots(4, 4, figsize=(12, 12))
    axes = axes.ravel()

    for i, idx in enumerate(random_indices):
        axes[i].imshow(sample_images[idx].squeeze(), cmap='gray')
        
        true_label = class_labels[sample_true_classes[idx]]
        pred_label = class_labels[sample_pred_classes[idx]]
        confidence = sample_preds[idx][sample_pred_classes[idx]] * 100

        # Color: green if correct, red if wrong
        color = 'green' if sample_true_classes[idx] == sample_pred_classes[idx] else 'red'
        
        axes[i].set_title(f'True: {true_label}\nPred: {pred_label} ({confidence:.1f}%)', 
                        color=color, fontweight='bold')
        axes[i].axis('off')

    plt.suptitle('Sample Predictions on Validation Set', fontsize=16, fontweight='bold')
    plt.tight_layout()
    #plt.show()
    return plt

In [ ]:
def plot_reports():

    class_labels = ['Angry', 'Disgust', 'Fear', 'Happy', 'Neutral', 'Sad', 'Surprise']

    publish_final_evaluation (train_acc,val_acc,train_loss,val_loss)

    #plot_training_curve_head = plot_training_curve_head(train_set,val_acc, train_loss, val_loss)
    #plot_training_curve_head.savefig(os.path.join(output_dir, 'training_curve_head.png'), dpi=300, bbox_inches='tight')
    #plot_training_curve_head.show()

    training_history_plt=plot_training_history(history_head, history_ft)
    training_history_plt.savefig(os.path.join(output_dir, 'training_history.png'), dpi=300, bbox_inches='tight')
    training_history_plt.show

    per_class_plt=generate_per_class_report(model,history_ft, validation_set,class_labels,output_dir)
    per_class_plt.savefig(os.path.join(output_dir, 'training_history.png'), dpi=300, bbox_inches='tight')
    per_class_plt.show

    cm_plt,cm_norm_plt,cm, cm_norm=generate_confusion_matrix_reports(history_ft, validation_set,class_labels)
    cm_plt.savefig(os.path.join(output_dir, 'confusion_matrix.png'), dpi=300, bbox_inches='tight')
    cm_norm_plt.savefig(os.path.join(output_dir, 'confusion_matrix.png'), dpi=300, bbox_inches='tight')
    cm_plt.show
    cm_norm_plt.show

    random_predictions_plt=test_random_predictions(validation_set,class_labels)
    random_predictions_plt.savefig(os.path.join(output_dir, 'random_predictions.png'), dpi=300, bbox_inches='tight')
    random_predictions_plt.show
    return

In [ ]:
def save_plot(plot, output_dir):
    if getattr(plot, "_suptitle", None) is not None:
        plot_name = plot._suptitle.get_text()
    else:
        axes=plot.get_axes()
        if axes:
            plot_name = axes[0].get_title()
    
    
    out_path = Path(output_dir) / plot_name
    try:
        plt.savefig(os.path.join(out_path,f"{plot_name}.png"),dpi=300,bbox_inches='tight')
        print(f"\n💾 Plot saved: {out_path}")
    except Exception as e:
        print(f"Error saving plot: {e}")
    
    return out_path

In [ ]:
# =========================================================================
# Enhanced Model Evaluation and Reporting
# =========================================================================

class ModelEvaluator:
    """Comprehensive model evaluation and report generation"""
    
    def __init__(self, model, test_set, class_labels, output_dir=None, history_hd=None, history_ft=None):
        self.model = model
        self.test_set = test_set
        self.class_labels = class_labels
        self.output_dir = output_dir
        self.history_hd = history_hd  
        self.history_ft = history_ft
        
        # Initialize prediction attributes
        self.predictions = None
        self.true_labels = None
        self.pred_labels = None
        
        
        
    def generate_predictions(self):
        """Generate predictions on test set"""
        print("🔄 Generating predictions...")
        self.test_set.reset()
        
        self.predictions = self.model.predict(self.test_set, verbose=1)
        self.true_labels = self.test_set.classes
        self.pred_labels = np.argmax(self.predictions, axis=1)
        
        return self.predictions, self.true_labels
    
    def create_comprehensive_report(self):
        """Generate comprehensive evaluation report"""
        if self.predictions is None:
            self.generate_predictions()
        
          
        # Generate all reports
        self.plot_training_history()
        self.plot_confusion_matrix()
        self.plot_normalized_confusion_matrix()
        self.generate_classification_report()
        self.plot_per_class_metrics()
        self.plot_roc_curves()
        self.plot_precision_recall_curves()
        self.generate_error_analysis()
        self.create_summary_report()
    
    """
    Traning history chart will only get returned if training history in memeory
    """    
    def plot_training_history(self):
        """Plot training history with head and fine-tune stages"""
        if self.history_hd is None and self.history_ft is None:
            print("⚠️ No training history provided")
            return None
            
        # Combine histories
        def _concat_metric(metric):
            vals = []
            if self.history_hd is not None:
                vals += self.history_hd.history.get(metric, [])
            if self.history_ft is not None:
                vals += self.history_ft.history.get(metric, [])
            return vals

        train_acc = _concat_metric('accuracy')
        val_acc = _concat_metric('val_accuracy')
        train_loss = _concat_metric('loss')
        val_loss = _concat_metric('val_loss')
        
        if not any([train_acc, val_acc, train_loss, val_loss]):
            print("⚠️ No training metrics found in history")
            return None
            
        fig, axes = plt.subplots(1, 2, figsize=(15, 5))
        
        # Accuracy plot
        if train_acc:
            axes[0].plot(train_acc, label='Train Accuracy', linewidth=2, color='blue')
        if val_acc:
            axes[0].plot(val_acc, label='Validation Accuracy', linewidth=2, color='red')
        axes[0].set_title('Model Accuracy Over Training')
        axes[0].set_xlabel('Epoch')
        axes[0].set_ylabel('Accuracy')
        axes[0].legend()
        axes[0].grid(True, alpha=0.3)
        
        # Loss plot
        if train_loss:
            axes[1].plot(train_loss, label='Train Loss', linewidth=2, color='blue')
        if val_loss:
            axes[1].plot(val_loss, label='Validation Loss', linewidth=2, color='red')
        axes[1].set_title('Model Loss Over Training')
        axes[1].set_xlabel('Epoch')
        axes[1].set_ylabel('Loss')
        axes[1].legend()
        axes[1].grid(True, alpha=0.3)
        
        # Add vertical line to show fine-tuning start if applicable
        if self.history_hd is not None and self.history_ft is not None:
            hd_epochs = len(self.history_hd.history.get('loss', []))
            axes[0].axvline(x=hd_epochs, color='green', linestyle='--', alpha=0.7, label='Fine-tune start')
            axes[1].axvline(x=hd_epochs, color='green', linestyle='--', alpha=0.7, label='Fine-tune start')
            axes[0].legend()
            axes[1].legend()
        
        plt.tight_layout()
        
        return plt
    
    def plot_confusion_matrix(self):
        """Plot confusion matrix"""
        cm = confusion_matrix(self.true_labels, self.pred_labels)
        
        plt.figure(figsize=(6, 4))
        sns.heatmap(cm, annot=True, fmt='d', cmap='Blues',
                   xticklabels=self.class_labels,
                   yticklabels=self.class_labels)
        plt.title('Confusion Matrix')
        plt.xlabel('Predicted')
        plt.ylabel('Actual')
        plt.tight_layout()
        return plt
        
    def plot_normalized_confusion_matrix(self):
        """Plot normalized confusion matrix"""
        cm = confusion_matrix(self.true_labels, self.pred_labels)
        cm_normalized = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]
        
        plt.figure(figsize=(6, 4))
        sns.heatmap(cm_normalized, annot=True, fmt='.2%', cmap='Blues',
                   xticklabels=self.class_labels,
                   yticklabels=self.class_labels)
        plt.title('Confusion Matrix-Normalized')
        plt.xlabel('Predicted')
        plt.ylabel('Actual')
        plt.tight_layout()
        return plt
        
    def generate_classification_report(self):
        """Generate detailed classification report"""
        report = classification_report(
            self.true_labels, 
            self.pred_labels,
            target_names=self.class_labels,
            output_dict=True
        )
        
        # Save as JSON
        with open(os.path.join(self.output_dir, 'classification_report.json'), 'w') as f:
            json.dump(report, f, indent=2)
            
        # Save as CSV
        report_df = pd.DataFrame(report).transpose()
        report_df.to_csv(os.path.join(self.output_dir, 'classification_report.csv'))
        
        # Print summary
        print("\n📊 Classification Report:")
        print(classification_report(
            self.true_labels, 
            self.pred_labels,
            target_names=self.class_labels
        ))
        
    def plot_per_class_metrics(self):
        """Plot precision, recall, and F1-score per class"""
        precision, recall, f1, support = precision_recall_fscore_support(
            self.true_labels, self.pred_labels, average=None
        )
        
        x = np.arange(len(self.class_labels))
        width = 0.25
        
        fig, ax = plt.subplots(figsize=(6, 4))
        bars1 = ax.bar(x - width, precision, width, label='Precision', alpha=0.8)
        bars2 = ax.bar(x, recall, width, label='Recall', alpha=0.8)
        bars3 = ax.bar(x + width, f1, width, label='F1-Score', alpha=0.8)
        
        ax.set_xlabel('Emotion Classes')
        ax.set_ylabel('Score')
        ax.set_title('Per-Class Performance Metrics')
        ax.set_xticks(x)
        ax.set_xticklabels(self.class_labels, rotation=45)
        ax.legend()
        ax.grid(True, alpha=0.3)
        
        # Add value labels
        for bars in [bars1, bars2, bars3]:
            for bar in bars:
                height = bar.get_height()
                ax.text(bar.get_x() + bar.get_width()/2., height,
                       f'{height:.2f}', ha='center', va='bottom')
        
        plt.tight_layout()
        plt_name = 'per_class_metrics.png'
        return plt
        
    def plot_roc_curves(self):
        """Plot ROC curves for multi-class classification"""
        from sklearn.preprocessing import label_binarize
        from sklearn.metrics import roc_curve, auc
        
        # Binarize labels
        y_test_bin = label_binarize(self.true_labels, classes=range(len(self.class_labels)))
        
        # Compute ROC curve and ROC area for each class
        fpr = dict()
        tpr = dict()
        roc_auc = dict()
        
        for i in range(len(self.class_labels)):
            fpr[i], tpr[i], _ = roc_curve(y_test_bin[:, i], self.predictions[:, i])
            roc_auc[i] = auc(fpr[i], tpr[i])
        
        # Plot
        plt.figure(figsize=(6, 4))
        colors = plt.cm.tab10(np.linspace(0, 1, len(self.class_labels)))
        
        for i, color in zip(range(len(self.class_labels)), colors):
            plt.plot(fpr[i], tpr[i], color=color, lw=2,
                    label=f'{self.class_labels[i]} (AUC = {roc_auc[i]:0.2f})')
        
        plt.plot([0, 1], [0, 1], 'k--', lw=2)
        plt.xlim([0.0, 1.0])
        plt.ylim([0.0, 1.05])
        plt.xlabel('False Positive Rate')
        plt.ylabel('True Positive Rate')
        plt.title('ROC Curves-Multi-class')
        plt.legend(loc="lower right")
        plt.grid(True, alpha=0.3)
        plt.tight_layout()
        return plt
        
    
    def plot_precision_recall_curves(self):
        """Plot precision-recall curves for multi-class classification"""
        from sklearn.preprocessing import label_binarize
        from sklearn.metrics import precision_recall_curve, average_precision_score
        
        # Binarize labels
        y_test_bin = label_binarize(self.true_labels, classes=range(len(self.class_labels)))
        
        # Compute precision-recall curve and average precision for each class
        precision = dict()
        recall = dict()
        average_precision = dict()
        
        for i in range(len(self.class_labels)):
            precision[i], recall[i], _ = precision_recall_curve(
                y_test_bin[:, i], self.predictions[:, i]
            )
            average_precision[i] = average_precision_score(
                y_test_bin[:, i], self.predictions[:, i]
            )
        
        # Plot
        plt.figure(figsize=(6, 4))
        colors = plt.cm.tab10(np.linspace(0, 1, len(self.class_labels)))
        
        for i, color in zip(range(len(self.class_labels)), colors):
            plt.plot(recall[i], precision[i], color=color, lw=2,
                    label=f'{self.class_labels[i]} (AP = {average_precision[i]:0.2f})')
        
        plt.xlim([0.0, 1.0])
        plt.ylim([0.0, 1.05])
        plt.xlabel('Recall')
        plt.ylabel('Precision')
        plt.title('Precision-Recall Curves-Multi-class')
        plt.legend(loc="best")
        plt.grid(True, alpha=0.3)
        plt.tight_layout()
        plt_prc=plt
        
        
        # Also create micro-average precision-recall curve
        precision["micro"], recall["micro"], _ = precision_recall_curve(
            y_test_bin.ravel(), self.predictions.ravel()
        )
        average_precision["micro"] = average_precision_score(
            y_test_bin, self.predictions, average="micro"
        )
        
        # Plot micro-average
        plt.figure(figsize=(6, 4))
        plt.plot(recall["micro"], precision["micro"], color='gold', lw=2,
                label=f'Micro-average (AP = {average_precision["micro"]:0.2f})')
        
        plt.xlim([0.0, 1.0])
        plt.ylim([0.0, 1.05])
        plt.xlabel('Recall')
        plt.ylabel('Precision')
        plt.title('Precision-Recall Curve-Micro-average')
        plt.legend(loc="best")
        plt.grid(True, alpha=0.3)
        plt.tight_layout()
        plt_prm=plt
        
        return plt_prm, plt_prc      
      
    def generate_error_analysis(self):
        """Analyze misclassifications"""
        errors = self.true_labels != self.pred_labels
        error_indices = np.where(errors)[0]
        
        if len(error_indices) > 0:
            # Get error details
            error_details = []
            for idx in error_indices[:50]:  # Limit to first 50 errors
                true_class = self.class_labels[self.true_labels[idx]]
                pred_class = self.class_labels[self.pred_labels[idx]]
                confidence = self.predictions[idx][self.pred_labels[idx]]
                
                error_details.append({
                    'index': idx,
                    'true_class': true_class,
                    'pred_class': pred_class,
                    'confidence': confidence,
                    'image_path': self.test_set.filepaths[idx] if hasattr(self.test_set, 'filepaths') else None
                })
            
            # Save error analysis
            error_df = pd.DataFrame(error_details)
            error_df.to_csv(os.path.join(self.output_dir, 'error_analysis.csv'), index=False)
            
            # Print top misclassifications
            print("\n🔍 Top 5 Misclassifications:")
            print("="*50)
            for _, row in error_df.head().iterrows():
                print(f"True: {row['true_class']} → Pred: {row['pred_class']} "
                      f"(Confidence: {row['confidence']:.2%})")
                        
    def create_summary_report(self):
        """Create comprehensive summary report"""
        summary = {
            'model_name': self.model.name,
            'test_samples': len(self.true_labels),
            'overall_accuracy': np.mean(self.true_labels == self.pred_labels),
            'class_labels': self.class_labels,
            'timestamp': datetime.now().isoformat()
        }
        
        # Add per-class metrics
        precision, recall, f1, support = precision_recall_fscore_support(
            self.true_labels, self.pred_labels, average=None
        )
        
        summary['per_class_metrics'] = {
            label: {
                'precision': float(p),
                'recall': float(r),
                'f1_score': float(f),
                'support': int(s)
            }
            for label, p, r, f, s in zip(self.class_labels, precision, recall, f1, support)
        }
        
        # Save summary
        with open(os.path.join(self.output_dir, 'summary_report.json'), 'w') as f:
            json.dump(summary, f, indent=2)
            
        return summary

In [ ]:
def run_evaluation(config, train_set, validation_set, class_labels, data_path):
    
    
    # Load model
    model_path_ft, model_path_hd, output_dir = load_model_path()
    model_ft, _ = load_models_from_file(model_path_hd, model_path_ft)
    
    
    # Run evaluation
    evaluator = ModelEvaluator(model_ft, validation_set, class_labels, output_dir)
    predictions, true_labels = evaluator.generate_predictions()
    
    plt_train_hist=evaluator.plot_training_history()
    plt_cm= evaluator.plot_confusion_matrix()
    plt_cm_norm = evaluator.plot_normalized_confusion_matrix()
    plt_prm, plt_prc=evaluator.plot_precision_recall_curves()
    plt_class= evaluator.plot_per_class_metrics()
    plt_roc= evaluator.plot_roc_curves()
    
    plots=[plt_train_hist, plt_cm, plt_cm_norm, plt_prm, plt_prc, plt_class, plt_roc]
    
    
    for fig in plots:
        fig.show
        #save_plot(fig, output_dir)
        fig.close
    
    #evaluator.create_comprehensive_report()
    

    random_predictions_plt= test_random_predictions(model_ft,validation_set, class_labels)
    random_predictions_plt.show
    print(f"✅ Evaluation complete! Reports saved to: {output_dir}")
    
    return evaluator

In [ ]:
def train_model(config, train_set, validation_set):

    # =========================================================================
    # Load Data & Compile Model  
    # =========================================================================

    # Generate training and validation Set
    print("Train image shape:", train_set.image_shape)
    print("Val image shape:  ", validation_set.image_shape)

    print(f"\n✅ Data loaded successfully")
    print (f"Colour Mode: {config.config.get('color_mode')}")    
    print(f"Training samples: {train_set.n}")
    print(f"Validation samples: {validation_set.n}")
    print(f"Class indices: {train_set.class_indices}")

    # Build model
    model, base_model = build_model(config)

    # Validate shapes
    assert train_set.image_shape == model.input_shape[1:], (
        f"Shape mismatch: {train_set.image_shape} vs {model.input_shape[1:]}")

    # Compile with optimizer choice
    optimizer_name =  "adam"
    if optimizer_name == "adam":
        opt = Adam(learning_rate=config.get('learning_rate'))
    elif optimizer_name == "adamw":
        opt = tf.keras.optimizers.AdamW(
            opt = Adam(learning_rate=config.get('learningrate')),
            weight_decay=config.get("weight_decay", 1e-4)
        )
    elif optimizer_name == "sgd":
        opt = tf.keras.optimizers.SGD(
            learning_rate=config.get('learningrate'),
            momentum=0.9,
            nesterov=True
        )
    
    model.compile(
        optimizer=opt,
        loss='categorical_crossentropy',
        metrics=['accuracy']
    )
    
    #print ("== Base Model ====")
    #base_model.summary()
    print ("== Full Model ====")
    model.summary()

    if config.get("use_class_weights"):
        class_weights = compute_class_weights(train_set)

    # Callbacks
    callbacks = create_callbacks(config)
    print(f"✅ Callbacks configured: {len(callbacks)} callbacks")
    

    # Training
    print(f"Training {config.backbone} model...")
    history_hd= train_stage1(config, model, train_set, validation_set, class_weights, callbacks)
    
    model.compile(
        optimizer=opt,
        loss='categorical_crossentropy',
        metrics=['accuracy']
    )
    
    #print ("== Base Model ====")
    #base_model.summary()
    print ("== Full Model ====")
    model.summary()
    
    # Stage 2: Fine-tune (if transfer learning)    
    history_ft=train_stage2(config, base_model, model, train_set, validation_set, class_weights, callbacks)
    

    print(f"\n{'='*70}")
    print(f"Building model with '{config.get('backbone')}' backbone...")
    print(f"{'='*70}")
    
    return model, history_hd, history_ft

In [ ]:
# Run 
if __name__ == "__main__":
    # setup
    config = ModelConfig("mobilenet")  # or 'custom_cnn', 'efficientnet'
    set_host_config_overides (config=config,dev=True)
    data_path=get_data_path()
    train_set, validation_set=create_data_generators(config, data_path)
    class_labels = ['Angry', 'Disgust', 'Fear', 'Happy', 'Neutral', 'Sad', 'Surprise']
    
    model, history_hd, history_ft = train_model(config, train_set, validation_set)
    


In [ ]:
plt = plot_training_history(history_hd, history_ft)
plt.show
    


In [ ]:
# =========================================================================
# Extract Values from the Model
# =========================================================================


# Helper to safely get values
def get_vals(h, key):
    return h.history.get(key, []) if h is not None else []

head_acc = get_vals(history_hd, 'val_accuracy')
head_val_acc = get_vals(history_hd, 'val_accuracy')
head_loss = get_vals(history_hd, 'loss')
head_val_loss = get_vals(history_hd, 'val_loss')

ft_acc = get_vals(history_ft, 'accuracy')
ft_val_acc = get_vals(history_ft, 'val_accuracy')
ft_loss = get_vals(history_ft, 'loss')
ft_val_loss = get_vals(history_ft, 'val_loss')

train_acc = head_acc + ft_acc
val_acc   = head_val_acc + ft_val_acc
train_loss = head_loss + ft_loss

In [ ]:

# helper: try several metric key variants
def pick_metric(h, names):
    if h is None: return []
    for n in names:
        if n in h.history:
            return h.history[n]
    return []

train_acc = pick_metric(history_hd, ['accuracy', 'acc'])
val_acc   = pick_metric(history_hd, ['val_accuracy', 'val_acc'])
# if fine-tune added, append those too
train_acc += pick_metric(history_ft, ['accuracy', 'acc'])
val_acc   += pick_metric(history_ft, ['val_accuracy', 'val_acc'])

train_loss = pick_metric(history_hd, ['loss'])
val_loss   = pick_metric(history_hd, ['val_loss'])
train_loss += pick_metric(history_ft, ['loss'])
val_loss   += pick_metric(history_ft, ['val_loss'])

# warn if empty
if not any([train_acc, val_acc, train_loss, val_loss]):
    print("⚠️ No training metrics found. Check history objects and metric names. Available keys:")
    if history_hd: print("head keys:", list(history_hd.history.keys()))
    if history_ft: print("ft keys:  ", list(history_ft.history.keys()))
    

    # create x-axis explicitly
    epochs = max(len(train_acc), len(val_acc), len(train_loss), len(val_loss))
    x = range(1, epochs + 1)

    fig, axes = plt.subplots(1, 2, figsize=(14, 5))
    if train_acc or val_acc:
        axes[0].plot(x[:len(train_acc)], train_acc, label='Train Accuracy', linewidth=2)
        axes[0].plot(x[:len(val_acc)], val_acc, label='Val Accuracy', linewidth=2)
        axes[0].set_xlabel('Epoch'); axes[0].set_ylabel('Accuracy'); axes[0].legend(); axes[0].grid(True)
    if train_loss or val_loss:
        axes[1].plot(x[:len(train_loss)], train_loss, label='Train Loss', linewidth=2)
        axes[1].plot(x[:len(val_loss)], val_loss, label='Val Loss', linewidth=2)
        axes[1].set_xlabel('Epoch'); axes[1].set_ylabel('Loss'); axes[1].legend(); axes[1].grid(True)

    plt.tight_layout()
    plt.show()   # <-- call it

In [ ]:
    
    
    #print("Global policy:", mixed_precision.global_policy())
    #print("Model dtype policy:", model.dtype_policy)   # or model.input_dtype / model.output_dtype  
    
    #evaluator = run_evaluation()
